# 接入大模型多轮对话

同济子豪兄 2024-6-12

## 安装所需工具包

In [16]:
!pip install openai qianfan --upgrade

## 导入封装好的语音合成函数

In [1]:
from zihao_tts import *

INFO:ChatTTS.ChatTTS.core:Load from cache: /home/featurize/.cache/huggingface/hub/models--2Noise--ChatTTS/snapshots/cc14302f34d7855eb3420d1fd48345012ff1460d
INFO:ChatTTS.ChatTTS.core:use cuda:0
INFO:ChatTTS.ChatTTS.core:vocos loaded.
INFO:ChatTTS.ChatTTS.core:dvae loaded.
INFO:ChatTTS.ChatTTS.core:gpt loaded.
INFO:ChatTTS.ChatTTS.core:decoder loaded.
INFO:ChatTTS.ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.ChatTTS.core:All initialized.


## 零一万物大语言模型

零一万物大模型开放平台：https://platform.lingyiwanwu.com

In [3]:
import openai
from openai import OpenAI

In [4]:
# 选择大模型
# https://platform.lingyiwanwu.com/docs

MODEL = 'yi-large'
# MODEL = 'yi-medium'
# MODEL = 'yi-spark'

In [ ]:
# 零一万物大模型开放平台
# https://platform.lingyiwanwu.com
YI_KEY = "f8144ffaff7c459791XXXXXXXXX"

In [5]:
client = OpenAI(
    api_key=YI_KEY,
    base_url='https://api.lingyiwanwu.com/v1'
)

In [14]:
# 构造多轮对话上下文

# 初始提示词
PROMPT = '''
你把我下面要说的话，转述成情商更高的另一句话，避免冒犯和冲突。并加入代表笑的[laugh]、代表停顿的[uv_break]，这两个语气词前后都要加空格。
你的回复要尽量简短，10个字以内。
以下是一些样例回复：
我说：你怎么这么笨。你说：你怎么这么可爱 [laugh]
我说：你穿这件衣服好丑。你说：这件衣服 [uv_break] 配不上有气质的你
我说：学这么久还没学会？你说：我可以慢慢教你。
我说：这个班是一天都上不下去了。你说：老板的想法 [laugh] 真的很棒。
我说：你怎么不回我消息。你说：我知道你在忙，忙完告诉我。
我说：这高考成绩只能读大专。你说：有考虑做说唱歌手吗？
我说：。你说：你不是我喜欢的类型。

我现在要说的话是：你长得太丑了，我们不可能在一起的
'''

# 多轮对话上下文，人类和AI一问一答，第一句和最后一句是人类说的
MESSAGES = [
        {'role': 'user', 'content': PROMPT},
        {'role': 'assistant', 'content': '你不是我喜欢的类型 [laugh]'},
        {'role': 'user', 'content': '写论文真是要了我的老命'},
    ]

In [8]:
# 大模型推理
completion = client.chat.completions.create(
    model=MODEL,
    messages=MESSAGES
)

# 解析大模型回复
result = completion.choices[0].message.content.strip()

INFO:httpx:HTTP Request: POST https://api.lingyiwanwu.com/v1/chat/completions "HTTP/1.1 200 OK"


In [10]:
result

'写论文 [uv_break] 真不容易 [laugh]'

In [11]:
# ChatTTS语音合成
wavs = zihao_tts(result, oral=2, laugh=9, bk=3)

INFO:ChatTTS.ChatTTS.core:All initialized.
  7%|▋         | 145/2048 [00:02<00:33, 57.16it/s]


In [12]:
# 试听
Audio(wavs[0], rate=24000)

## 百度智能云千帆ModelBuilder

In [ ]:
# 百度智能云千帆ModelBuilder
# https://qianfan.cloud.baidu.com
QIANFAN_ACCESS_KEY = "ALTAKRELRxSXXXXXXXXXX"
QIANFAN_SECRET_KEY = "3737d9da82de4f2XXXXXXXXXX"

In [3]:
import os
import qianfan

# 传入 ACCESS_KEY 和 SECRET_KEY
os.environ["QIANFAN_ACCESS_KEY"] = QIANFAN_ACCESS_KEY
os.environ["QIANFAN_SECRET_KEY"] = QIANFAN_SECRET_KEY

In [5]:
# 选择大语言模型
MODEL = "ERNIE-Bot-4"
# MODEL = "ERNIE Speed"
# MODEL = "ERNIE-Lite-8K"
# MODEL = 'ERNIE-Tiny-8K'

In [6]:
chat_comp = qianfan.ChatCompletion(model=MODEL)

In [7]:
# 初始提示词

PROMPT = '''
你把我下面要说的话，转述成情商更高的另一句话，避免冒犯和冲突。并加入代表笑的[laugh]、代表停顿的[uv_break]，这两个语气词前后都要加空格。
你的回复要尽量简短，10个字以内。
以下是一些样例回复：
我说：你怎么这么笨。你说：你怎么这么可爱 [laugh]
我说：你穿这件衣服好丑。你说：这件衣服 [uv_break] 配不上有气质的你
我说：学这么久还没学会？你说：我可以慢慢教你。
我说：这个班是一天都上不下去了。你说：老板的想法 [laugh] 真的很棒。
我说：你怎么不回我消息。你说：我知道你在忙，忙完告诉我。
我说：这高考成绩只能读大专。你说：有考虑做说唱歌手吗？
我说：。你说：你不是我喜欢的类型。

我现在要说的话是：你长得太丑了，我们不可能在一起的
'''

# 多轮对话上下文，人类和AI一问一答，第一句和最后一句是人类说的

MESSAGES = [
        {'role': 'user', 'content': PROMPT},
        {'role': 'assistant', 'content': '你 [uv_break] 不是我喜欢的类型 [laugh]'},
        {'role': 'user', 'content': '这个班是一天也上不下去了'},
    ]

In [9]:
# 输入给大模型
resp = chat_comp.do(
    messages=MESSAGES,
    top_p=0.8, 
    temperature=0.3, 
    penalty_score=1.0
)

result = resp["result"]

In [11]:
result

'这个班 [uv_break] 有点待不住了啊[laugh]'

- ChatTTS语音合成

In [12]:
# ChatTTS语音合成
wavs = zihao_tts(result, oral=2, laugh=9, bk=3)

INFO:ChatTTS.ChatTTS.core:All initialized.
  8%|▊         | 161/2048 [00:02<00:32, 58.39it/s]


In [13]:
# 试听
Audio(wavs[0], rate=24000)